In [102]:
from mypulp import *
import numpy as np
import pandas as pd

In [103]:
p = {}
p[0] = 5
p[1] = 1
p[2] = 1

p[3] = 3
p[4] = 1

p[5] = 4
p[6] = 1

p[7] = 3
p[8] = 2

p[9] = 1
p[10] = 1
p[11] = 1

p[12] = 2
p[13] = 2
p[14] = 2

p[15] = 4
p[16] = 1
p[17] = 1

In [104]:
# Deine Model
model = Model("Cooking")

# Define Variable
feature_num = len(p)

x = {}# if job j starts before job k => 1
for j in range(feature_num):
    for k in range(feature_num):
        if j != k:
            x[j, k] = model.addVar(vtype="B")
    
s = {}# job j's start time
for j in range(feature_num):
    s[j] = model.addVar(vtype="C")

model.update()

In [105]:
# Disjunctive Constraint
for i in range(feature_num):
    for j in range(feature_num):
        if i != j:
            model.addConstr( s[i] + p[i] - s[j] <=     100*(1-x[i, j]))

In [106]:
for j in range(feature_num):
    for k in range(feature_num):
        if j < k:
            model.addConstr( x[j, k] + x[k, j] == 1)

In [107]:
#Start Time Constraint
for j in range(feature_num):
    model.addConstr( quicksum(p[k]*x[k , j] for k in range(feature_num) if j != k)  <= s[j])

In [108]:
# In the same redipe, it can't pass post-job
list_ = [0, 1, 3, 5, 7, 12, 13]

for i in list_:
    model.addConstr( s[i] + p[i]  <= s[i+1] )

# when using grill or frying pan, for the safe, heating time is strictly obeyed
model.addConstr( s[9] + p[9] + 5 == s[10] )
model.addConstr( s[10] + p[10] + 3 == s[11] )

model.addConstr( s[15] + p[15] + 4 == s[16] )
model.addConstr( s[16] + p[16] + 3 == s[17] )

# As Hotcook is shared with Steamed Sweet Potatoes and Roast Beef, so it can't use simultaneously
model.addConstr(  s[4] + p[4] + 15 - s[14]   <=  100*(1-x[4, 14]) )
model.addConstr(  s[14] + p[14] + 30 - s[4]  <=  100*(1-x[14, 4]) )

# As Frying pan is shared with Roast Beef and Pancake, so it can't use simultaneously
model.addConstr(  s[13] + p[13] +  - s[17]   <=  100*(1-x[13, 17]) )
model.addConstr(  s[17] + p[17] +  - s[13]  <=  100*(1-x[17, 13]) )

100*x_303 + -1*x_320 + 1*x_324 + -99 <= 0

In [109]:
model.setObjective(quicksum(s[j] for j in range(feature_num)), GRB.MINIMIZE )

model.optimize()

In [110]:
print(model.Status)

1


In [111]:
if model.Status == 1 :
    print('Opt. Value=', model.ObjVal)

    S_ = []
    for j in range(feature_num):
        S_.append([j, s[j].X])

Opt. Value= 379.0


In [112]:
dict_ = {"0":"0. French Fries:Peel&Cut", "1":"1. French Fries:set the material to the device", "2":"2. French Fries:clean up the sink", 
         "3":"3. Steamed Sweet Potatoes:Cut", "4":"4. Steamed Sweet Potatoes:set the material to the device",
         "5":"5. Steamed Eggplant:Cut", "6":"6. Steamed Eggplant:set the material to the device",
         "7":"7. Miso Soup:Peel&Cut", "8":"8. Miso Soup:set the material to the device", 
         "9":"9. Grilled Fish:set the material to the device", "10":"10. Grilled Fish:Turn over", "11":"11. Grilled Fish:Extinguish",
         "12":"12. Roast Beef:make a seasoning", "13":"13. Roast Beef:heating seasoning", "14":"14. Roast Beef:set the material to the device",
         "15":"15. Pancake:preprocess material & set to device", "16":"16. Pancake:Turn over", "17":"17. Pancake:Extinguish"}

In [113]:
result = pd.DataFrame( np.hstack( [np.array(S_)[np.argsort(np.array(S_)[:, 1]), :], 
                          np.array( [p[i] for i in  np.array(S_)[np.argsort(np.array(S_)[:, 1]), :][:, 0]] ).reshape(-1, 1)]), 
            columns=["Job Index", "Start Time", "Unit Time"])

result["Finish Time"] = result["Start Time"] + result["Unit Time"]
result["Job Name"] = result["Job Index"].astype("int").apply(lambda x : dict_[str(x)])
result[["Job Name", "Start Time", "Finish Time", "Unit Time"]]

,Job Name,Start Time,Finish Time,Unit Time
0,3. Steamed Sweet Potatoes:Cut,0.0,3.0,3.0
1,12. Roast Beef:make a seasoning,3.0,5.0,2.0
2,15. Pancake:preprocess material & set to device,5.0,9.0,4.0
3,5. Steamed Eggplant:Cut,9.0,13.0,4.0
4,16. Pancake:Turn over,13.0,14.0,1.0
5,7. Miso Soup:Peel&Cut,14.0,17.0,3.0
6,17. Pancake:Extinguish,17.0,18.0,1.0
7,6. Steamed Eggplant:set the material to the de...,18.0,19.0,1.0
8,9. Grilled Fish:set the material to the device,19.0,20.0,1.0
9,0. French Fries:Peel&Cut,20.0,25.0,5.0


In [114]:
color = []
for i in range(len(result)):
    if "French Fries" in result["Job Name"].iat[i]:
        color.append(0)
    elif "Steamed Sweet Potatoes" in result["Job Name"].iat[i]:
        color.append(1)
    elif "Steamed Eggplant" in result["Job Name"].iat[i]:
        color.append(2)
    elif "Miso Soup" in result["Job Name"].iat[i]:
        color.append(3)
    elif "Grilled Fish" in result["Job Name"].iat[i]:
        color.append(4)
    elif "Roast Beef" in result["Job Name"].iat[i]:
        color.append(5)
    elif "Pancake" in result["Job Name"].iat[i]:
        color.append(6)

result["Color"] = color

In [115]:
import plotly.express as px
import plotly.io as pio
from datetime import datetime

In [116]:
result["Start Time DT"] = result["Start Time"].apply(lambda x : datetime(2021, 2, 9, 0, 0, int(x)))
result["Finish Time DT"] = result["Finish Time"].apply(lambda x : datetime(2021, 2, 9, 0, 0, int(x)))

In [120]:
fig = px.timeline(result, x_start="Start Time DT", x_end="Finish Time DT", y="Job Name", color="Color")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
pio.write_html(fig,file="./6-1.html")